# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298477915383                   -0.85    5.0
  2   -8.300215054539       -2.76       -1.26    1.4
  3   -8.300446563139       -3.64       -1.89    3.0
  4   -8.300462337984       -4.80       -2.74    2.1
  5   -8.300464532881       -5.66       -3.18    2.6
  6   -8.300464603637       -7.15       -3.34    1.4
  7   -8.300464624743       -7.68       -3.47   11.2
  8   -8.300464633329       -8.07       -3.59    1.0
  9   -8.300464641143       -8.11       -3.80    1.0
 10   -8.300464642338       -8.92       -3.89    2.1
 11   -8.300464643781       -8.84       -4.20    2.6


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67465612481                   -0.70    6.4
  2   -16.67871326109       -2.39       -1.14    1.4
  3   -16.67922879644       -3.29       -1.87    3.8
  4   -16.67928150361       -4.28       -2.78    4.2
  5   -16.67928607172       -5.34       -3.20    4.8
  6   -16.67928620806       -6.87       -3.49    3.6
  7   -16.67928622262       -7.84       -3.96    1.0
  8   -16.67928622420       -8.80       -4.63    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32732474153                   -0.56    7.1
  2   -33.33517663141       -2.11       -1.00    1.6
  3   -33.33673566198       -2.81       -1.72    7.0
  4   -33.33691573124       -3.74       -2.63    3.4
  5   -33.33694293991       -4.57       -3.13    5.9
  6   -33.33694388496       -6.02       -4.02    5.6


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298574666512                   -0.85    5.2
  2   -8.300275736247       -2.77       -1.59    1.0
  3   -8.300461016732       -3.73       -2.67    5.1
  4   -8.300442943853   +   -4.74       -2.60    8.9
  5   -8.300464511112       -4.67       -3.69    1.8
  6   -8.300464596455       -7.07       -3.97    4.1
  7   -8.300464642015       -7.34       -4.63    2.2


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32591973906                   -0.56    6.8
  2   -33.32851655068       -2.59       -1.28    1.0
  3   -25.57762730062   +    0.89       -0.62    7.5
  4   -33.20958265676        0.88       -1.49    7.5
  5   -33.28471207192       -1.12       -1.39    3.5
  6   -33.31281438584       -1.55       -1.81    3.6
  7   -32.77749618470   +   -0.27       -1.19    4.9
  8   -33.33389950956       -0.25       -2.24    4.9
  9   -33.33435981903       -3.34       -2.27    2.2
 10   -33.33608969335       -2.76       -2.42    2.1
 11   -33.33660816128       -3.29       -2.74    1.9
 12   -33.33691692096       -3.51       -3.18    2.9
 13   -33.33693588070       -4.72       -3.40    3.1
 14   -33.33694216601       -5.20       -3.62    2.5
 15   -33.33694216014   +   -8.23       -3.84    3.2
 16   -33.33694386369       -5.77       -4.36    2.6


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.